In [1]:
import pandas as pd
import numpy as np
import json
import requests
import time
from datetime import datetime

In [2]:
def getRouteNames(list_station_id):
    api_url_base = 'https://api-v3.mbta.com/'
    headers = {'Content-Type': 'application/json'}
    #routes_arg = 'routes?filter[type]=0,1'
    routes_arg = f'/schedules?include=route,stop&filter[stop]={list_station_id}&api_key=61f58a50028c468da41ce45382fae36f'
    resp = requests.get(api_url_base+routes_arg, headers=headers)
    if resp.status_code == 200:
        return resp
    print('Error: getRouteNames API endpoint')


# schedule_arg = 'routes?filter[type]=0,1'
# def getSchedule():
#     resp_schedule = requests.get(api_url_base+routes_arg, headers=headers)
#     if resp_schedule.status_code == 200:
#         return resp_schedule
#     print('Error: getSchedule API endpoint')

In [10]:
# Perform API request for MBTA Routes of type 0,1
# Print long_name of each route
# Return list of ids for routes to be used in other API requests
def routeIdList(list_station_id):
    route_id_list = []
    stop_id_list = []
    direction_id_list = []
    arrival_time_list = []
    departure_time_list = []
    df = pd.DataFrame()
    
    for stop in list_station_id:
        time.sleep(1)
        response_data = getRouteNames(stop)
        json_response_data = response_data.json()
        resp_data = json_response_data["data"]
        
        for data in resp_data:
            route_id = data["relationships"]["route"]["data"]["id"]
            stop_id = data["relationships"]["stop"]["data"]["id"]
            arrival_time = data["attributes"]["arrival_time"]
            departure_time = data["attributes"]["departure_time"]
            direction_id = data["attributes"]["direction_id"]
            #print(f"{route_id}\t{stop_id}\t{arrival_time}")
            route_id_list.append(route_id)
            stop_id_list.append(stop_id)
            arrival_time_list.append(arrival_time)
            departure_time_list.append(departure_time)
            direction_id_list.append(direction_id)
            #print('\n')
        # return list of route ids to be used in other queries
    df["RouteId"] = route_id_list
    df["StopId"] = stop_id_list
    df["ArrivalDateTime"] = arrival_time_list
    df["DepartureDateTime"] = departure_time_list
    df["DirectionId"] = direction_id_list   
    return df         
        
    
    

In [11]:
relevant_stops = ['70007','70240','70087','70265','70154','70258','70271','70091','70252','70104','70067','70199','70032','70030','70024','70063','70029','70027','70267','70177','70274','70158','70009','70096','70076','70257','70266','70015','70057','70044','70164','70100','70251','70245','70273','70183','70075','70156','70050','70247','70083','70092','70086','70167','70239','70170','70066','70035','70003','70008','70102','70173','70041','70242','70033','70042','70074','70203','70017','70098','70055','70165','70054','70005','70002','70244','70172','70186','70198','70016','70254','70264','70014','70039','70045','70278','70175','70081','70058','70077','70090','70048','70268','70022','70010','70182','70025','70034','70187','70026','70088','70082','70004','70168','70021','70006','70180','70162','70073','70069','70023','70243','70046','70011','70250','70255','70279','70079','70181','70169','70201','70246','70072','70179','70256','70166','70084','70089','70269','70080','70047','70019','70200','70078','70064','70049','70020','70053','70065','70028','70031','70241','70178','70068','70248','70253','70176','70085','70171','70040','70272','70071','70163','70070','70043','70174','70018','70056','70051','70013','70270','70249','70263','70052','70012']
df = pd.DataFrame()
df = routeIdList(relevant_stops)
df.head()

,RouteId,StopId,ArrivalDateTime,DepartureDateTime,DirectionId
0,Orange,70007,2021-02-08T00:18:00-05:00,2021-02-08T00:18:00-05:00,1
1,Orange,70007,2021-02-08T00:33:00-05:00,2021-02-08T00:33:00-05:00,1
2,Orange,70007,2021-02-07T09:42:00-05:00,2021-02-07T09:42:00-05:00,1
3,Orange,70007,2021-02-07T09:05:00-05:00,2021-02-07T09:05:00-05:00,1
4,Orange,70007,2021-02-07T08:40:00-05:00,2021-02-07T08:40:00-05:00,1


In [12]:
# Perform API request for MBTA Routes of type 0,1
# Print long_name of each route
# Return list of ids for routes to be used in other API requests
def getNames(list_station_id):    
    stop_id_list = []
    #platform_name_list = []
    stop_name_list = []
    #description_list = []
    line_name_list = []
    
    df = pd.DataFrame()

    for stop in list_station_id:
        time.sleep(1)
        response = getRouteNames(stop)
        json_resp = response.json()
        resp_data = json_resp["included"]
        for i, data in enumerate(resp_data):
            if(i == 0):
                stop_id = data["id"]
                stop_name = data["attributes"]["name"]
                stop_id_list.append(stop_id)
                stop_name_list.append(stop_name)                
            if(i == 1):
                line_name = data["id"]
                line_name_list.append(line_name) 

        # return list of route ids to be used in other queries
    df["StopId"] = stop_id_list
    df["StopName"] = stop_name_list
    df["LineName"] = line_name_list
     
    return df

In [13]:
df_name = getNames(relevant_stops)
df_name.head()

,StopId,StopName,LineName
0,70007,Jackson Square,Orange
1,70240,Prudential,Green-E
2,70087,Savin Hill,Red
3,70265,Butler,Mattapan
4,70154,Copley,Green-B


In [14]:
new_df = pd.merge(df, df_name, on="StopId", how="left")
new_df.dropna(inplace = True)
new_df.head()

,RouteId,StopId,ArrivalDateTime,DepartureDateTime,DirectionId,StopName,LineName
0,Orange,70007,2021-02-08T00:18:00-05:00,2021-02-08T00:18:00-05:00,1,Jackson Square,Orange
1,Orange,70007,2021-02-08T00:33:00-05:00,2021-02-08T00:33:00-05:00,1,Jackson Square,Orange
2,Orange,70007,2021-02-07T09:42:00-05:00,2021-02-07T09:42:00-05:00,1,Jackson Square,Orange
3,Orange,70007,2021-02-07T09:05:00-05:00,2021-02-07T09:05:00-05:00,1,Jackson Square,Orange
4,Orange,70007,2021-02-07T08:40:00-05:00,2021-02-07T08:40:00-05:00,1,Jackson Square,Orange


In [15]:
new_df["ArrivalDateTime"] = new_df["ArrivalDateTime"].apply(lambda x: x.replace("T", " "))
new_df["DepartureDateTime"] = new_df["DepartureDateTime"].apply(lambda x: x.replace("T", " "))
new_df.head()


,RouteId,StopId,ArrivalDateTime,DepartureDateTime,DirectionId,StopName,LineName
0,Orange,70007,2021-02-08 00:18:00-05:00,2021-02-08 00:18:00-05:00,1,Jackson Square,Orange
1,Orange,70007,2021-02-08 00:33:00-05:00,2021-02-08 00:33:00-05:00,1,Jackson Square,Orange
2,Orange,70007,2021-02-07 09:42:00-05:00,2021-02-07 09:42:00-05:00,1,Jackson Square,Orange
3,Orange,70007,2021-02-07 09:05:00-05:00,2021-02-07 09:05:00-05:00,1,Jackson Square,Orange
4,Orange,70007,2021-02-07 08:40:00-05:00,2021-02-07 08:40:00-05:00,1,Jackson Square,Orange


In [16]:
new_df["Arrival_Day"] = new_df["ArrivalDateTime"].apply(lambda x: x.split()[0])
new_df["Arrival_Time"] = new_df["ArrivalDateTime"].apply(lambda x: (x.split()[1]).split("-")[0])
new_df["Departure_Day"] = new_df["ArrivalDateTime"].apply(lambda x: x.split()[0])
new_df["Departure_Time"] = new_df["ArrivalDateTime"].apply(lambda x: (x.split()[1]).split("-")[0])
new_df["ArrivalDay"] = new_df["Arrival_Day"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d").day)
new_df.head()

,RouteId,StopId,ArrivalDateTime,DepartureDateTime,DirectionId,StopName,LineName,Arrival_Day,Arrival_Time,Departure_Day,Departure_Time,ArrivalDay
0,Orange,70007,2021-02-08 00:18:00-05:00,2021-02-08 00:18:00-05:00,1,Jackson Square,Orange,2021-02-08,00:18:00,2021-02-08,00:18:00,8
1,Orange,70007,2021-02-08 00:33:00-05:00,2021-02-08 00:33:00-05:00,1,Jackson Square,Orange,2021-02-08,00:33:00,2021-02-08,00:33:00,8
2,Orange,70007,2021-02-07 09:42:00-05:00,2021-02-07 09:42:00-05:00,1,Jackson Square,Orange,2021-02-07,09:42:00,2021-02-07,09:42:00,7
3,Orange,70007,2021-02-07 09:05:00-05:00,2021-02-07 09:05:00-05:00,1,Jackson Square,Orange,2021-02-07,09:05:00,2021-02-07,09:05:00,7
4,Orange,70007,2021-02-07 08:40:00-05:00,2021-02-07 08:40:00-05:00,1,Jackson Square,Orange,2021-02-07,08:40:00,2021-02-07,08:40:00,7


In [27]:
def get_busiest_station(station_name, day, time_period):
    dict1 = dict()
    start_time = int(time_period.split("-")[0])
    end_time = int(time_period.split("-")[1])-1
    for index, row in new_df.iterrows():
        new_time = int(row["Arrival_Time"].split(":")[0])
        if(station_name == str(row["StopName"]) and str(day) == str(row["ArrivalDay"]) and (start_time<=new_time<=end_time)):
            if(new_time not in dict1):
                dict1[new_time] = 1
            else:
                dict1[new_time] += 1
    key, value = max(dict1.items(), key=lambda x:x[1])
    print(f"The busiest time is between {key}:00 to {str(int(key)+1)}:00 with arrivals of upto: {value} trains")
    
    return start_time, end_time,station_name, day, key


In [28]:
# busiest 1-hour time period over all stations
start_time, end_time,station_name, day, key = get_busiest_station("Park Street", 7, "10-13")

The busiest time is between 12:00 to 13:00 with arrivals of upto: 42 trains


In [31]:
new_df["Arrival_Hours"] = new_df["Arrival_Time"].apply(lambda x: int(x.split(":")[0]))
flag =  (new_df["Arrival_Hours"] == key) & (new_df["StopName"] == station_name) &(new_df["ArrivalDay"] == day)
question_two_df = new_df[flag].copy()
question_two_df.sort_values(by=["Arrival_Day", "Arrival_Time"], inplace = True)
question_two_df.reset_index(inplace = True)

In [32]:
question_two_df.head()


,index,RouteId,StopId,ArrivalDateTime,DepartureDateTime,DirectionId,StopName,LineName,Arrival_Day,Arrival_Time,Departure_Day,Departure_Time,ArrivalDay,Arrival_Hours
0,7941,Green-D,70198,2021-02-07 12:01:00-05:00,2021-02-07 12:01:00-05:00,0,Park Street,Green-D,2021-02-07,12:01:00,2021-02-07,12:01:00,7,12
1,1356,Green-E,70199,2021-02-07 12:02:00-05:00,2021-02-07 12:02:00-05:00,0,Park Street,Green-E,2021-02-07,12:02:00,2021-02-07,12:02:00,7,12
2,2958,Red,70076,2021-02-07 12:04:00-05:00,2021-02-07 12:04:00-05:00,1,Park Street,Red,2021-02-07,12:04:00,2021-02-07,12:04:00,7,12
3,4167,Red,70075,2021-02-07 12:05:00-05:00,2021-02-07 12:05:00-05:00,0,Park Street,Red,2021-02-07,12:05:00,2021-02-07,12:05:00,7,12
4,14063,Green-E,70200,2021-02-07 12:05:00-05:00,2021-02-07 12:05:00-05:00,1,Park Street,Green-B,2021-02-07,12:05:00,2021-02-07,12:05:00,7,12


In [34]:
question_two_df['Arrival_Time'] = pd.to_datetime(question_two_df['Arrival_Time'],format= '%H:%M:%S' ).dt.time
question_two_df['Departure_Time'] = pd.to_datetime(question_two_df['Departure_Time'],format= '%H:%M:%S' ).dt.time 

for index, row in question_two_df.iterrows(): 

    d1 = row["Departure_Time"].hour - row["Arrival_Time"].hour 
    d2 = row["Departure_Time"].minute - row["Arrival_Time"].minute
    
    sum1 = (d1 * 60 * 60  + d2 * 60)
    if(sum1 == 0):
        question_two_df["Difference_Time"] = 60
    else:
        question_two_df["Difference_Time"] = sum1

In [35]:
question_two_df.head()


,index,RouteId,StopId,ArrivalDateTime,DepartureDateTime,DirectionId,StopName,LineName,Arrival_Day,Arrival_Time,Departure_Day,Departure_Time,ArrivalDay,Arrival_Hours,Difference_Time
0,7941,Green-D,70198,2021-02-07 12:01:00-05:00,2021-02-07 12:01:00-05:00,0,Park Street,Green-D,2021-02-07,12:01:00,2021-02-07,12:01:00,7,12,60
1,1356,Green-E,70199,2021-02-07 12:02:00-05:00,2021-02-07 12:02:00-05:00,0,Park Street,Green-E,2021-02-07,12:02:00,2021-02-07,12:02:00,7,12,60
2,2958,Red,70076,2021-02-07 12:04:00-05:00,2021-02-07 12:04:00-05:00,1,Park Street,Red,2021-02-07,12:04:00,2021-02-07,12:04:00,7,12,60
3,4167,Red,70075,2021-02-07 12:05:00-05:00,2021-02-07 12:05:00-05:00,0,Park Street,Red,2021-02-07,12:05:00,2021-02-07,12:05:00,7,12,60
4,14063,Green-E,70200,2021-02-07 12:05:00-05:00,2021-02-07 12:05:00-05:00,1,Park Street,Green-B,2021-02-07,12:05:00,2021-02-07,12:05:00,7,12,60


In [37]:
import math
# Python variables
# Constants will be modified in the course of evaluating your solution.  Feel free to modify to extend your own evalutation.

# Sales function
A=2
B=10
def salesOverTime(t_seconds):
    return max(math.floor(A*math.atan(t_seconds/B)),0) 

# Scenario
CARS_PER_TRAIN      =  8
T_bw_SameLineCars   = 10    # seconds to move between cars on the same line (same train or different train of same color in same direction)
T_bw_DifferentLine  = 45    # seconds to move between cars of trains that are either on different lines (color) or different directions of the same line.

relevant_stops = ['70007','70240','70087','70265','70154','70258','70271','70091','70252','70104','70067','70199','70032','70030','70024','70063','70029','70027','70267','70177','70274','70158','70009','70096','70076','70257','70266','70015','70057','70044','70164','70100','70251','70245','70273','70183','70075','70156','70050','70247','70083','70092','70086','70167','70239','70170','70066','70035','70003','70008','70102','70173','70041','70242','70033','70042','70074','70203','70017','70098','70055','70165','70054','70005','70002','70244','70172','70186','70198','70016','70254','70264','70014','70039','70045','70278','70175','70081','70058','70077','70090','70048','70268','70022','70010','70182','70025','70034','70187','70026','70088','70082','70004','70168','70021','70006','70180','70162','70073','70069','70023','70243','70046','70011','70250','70255','70279','70079','70181','70169','70201','70246','70072','70179','70256','70166','70084','70089','70269','70080','70047','70019','70200','70078','70064','70049','70020','70053','70065','70028','70031','70241','70178','70068','70248','70253','70176','70085','70171','70040','70272','70071','70163','70070','70043','70174','70018','70056','70051','70013','70270','70249','70263','70052','70012']
T_trainAtStation = 60       # seconds for train to remain at station if arrival/departure times are equal (0 duration)© 2021 GitHub, Inc.


In [39]:
curr_time = (question_two_df["Arrival_Time"][0]).hour * 3600 + (question_two_df["Arrival_Time"][0]).minute * 60 + (question_two_df["Arrival_Time"][0]).second
final_sales = 0
for i in range(len(question_two_df)-1):
    curr_arr_time = (question_two_df["Arrival_Time"][i]).hour * 3600 + (question_two_df["Arrival_Time"][i]).minute * 60 + (question_two_df["Arrival_Time"][i]).second
    next_arr_time = (question_two_df["Arrival_Time"][i+1]).hour * 3600 + (question_two_df["Arrival_Time"][i+1]).minute * 60 + (question_two_df["Arrival_Time"][i+1]).second
    curr_dep_time = 60 + (question_two_df["Departure_Time"][i]).hour * 3600 + (question_two_df["Departure_Time"][i]).minute * 60 + (question_two_df["Departure_Time"][i]).second
    sum_sales = 0
    temp_time = 0
    diff = next_arr_time - curr_arr_time
    curr_direction =question_two_df["DirectionId"][i]
    next_direction =question_two_df["DirectionId"][i+1]
    
    if(curr_arr_time == next_arr_time):
        curr_time += diff
        continue
    if(index == 0 or curr_direction == next_direction):
        while(curr_time<curr_dep_time):
            if(curr_time+6<curr_dep_time):
                curr_time += 6
            else:
                break
            sum_sales += salesOverTime(6)
            if(curr_time+10<curr_dep_time):
                curr_time += 10
            else:
                break
    else:
        curr_time += 45
        while(curr_time<curr_dep_time):
            if(curr_time+6<curr_dep_time):
                curr_time += 6
            else:
                break
            sum_sales += salesOverTime(6)
            if(curr_time+10<curr_dep_time):
                curr_time += 10
            else:
                break
                
                
    curr_time += (curr_dep_time-curr_time) 
    final_sales += sum_sales
    
print("Sale done in the busiest hour",final_sales)

Sale done in the busiest hour 172
